# Auto-Merging Retreival - Source Group A

In [13]:
import os

from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Settings,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    # Set global settings
    Settings.llm = llm
    Settings.embed_model = embed_model

    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)

    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine


In [ ]:
from llama_index.llms.openai import OpenAI 
from llama_index.core import Settings

from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./SourceGroupA/source12.pdf", "./SourceGroupA/source1.pdf", "./SourceGroupA/source11.pdf", "./SourceGroupA/source2.pdf", "./SourceGroupA/source3.pdf", "./SourceGroupA/source4.pdf", "./SourceGroupA/source5.pdf", "./SourceGroupA/source6.pdf", "./SourceGroupA/source7.pdf", "./SourceGroupA/source8.pdf", "./SourceGroupA/source9.pdf", "./SourceGroupA/source10.pdf"]
).load_data()


In [15]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5),
    save_dir="./merging_index_GroupA",
)


In [16]:
from llama_index.core.response.notebook_utils import display_response

query_engine = get_automerging_query_engine(index, similarity_top_k=6)

with open('./generated_questions.txt') as file:
    for line in file:
        question = line.strip()
        print(question)
        display_response(query_engine.query(question))

Are current policing methods effective in reducing crime in minority communities?
> Merging 1 nodes into parent node.
> Parent node id: c5e5f938-6b3b-43ba-a50b-48b698e887b2.
> Parent node text: There is no evidence of systematic anti-Black disparities in fatal shootings of unarmed citizens....



**`Final Response:`** Current policing methods may be effective in reducing crime in minority communities, as indicated by the comparison of various data sets that show consistent results when analyzing fatal police shootings. The research suggests that there is no systematic anti-Black disparities in these incidents, and when accounting for estimated rates of police exposure, the outcomes do not show significant disparities in either direction. This information implies that current policing methods may be playing a role in addressing crime in minority communities.

How does the use of facial recognition technology impact racial profiling?
> Merging 1 nodes into parent node.
> Parent node id: c5e5f938-6b3b-43ba-a50b-48b698e887b2.
> Parent node text: There is no evidence of systematic anti-Black disparities in fatal shootings of unarmed citizens....



**`Final Response:`** The use of facial recognition technology can potentially exacerbate racial profiling due to issues with accuracy and bias in the algorithms. This can lead to disproportionate targeting of certain racial groups, contributing to systemic discrimination and injustices in law enforcement practices.

Is there evidence to support claims of systemic racism within law enforcement agencies?
> Merging 1 nodes into parent node.
> Parent node id: c5e5f938-6b3b-43ba-a50b-48b698e887b2.
> Parent node text: There is no evidence of systematic anti-Black disparities in fatal shootings of unarmed citizens....



...

**`Final Response:`** Underreporting hate crimes on minority communities can lead to a lack of accurate data, which in turn can result in insufficient resources and support being allocated to address and prevent such crimes. This can perpetuate a cycle of discrimination, violence, and fear within minority communities, as well as hinder efforts to hold perpetrators accountable and provide adequate protection for those affected.

How do law enforcement agencies address allegations of racial bias in their practices?


**`Final Response:`** Law enforcement agencies address allegations of racial bias in their practices by conducting research and studies to analyze data on police interactions and use of force. They compare different data sets to assess for any potential bias, such as the Crime Victimization Survey and the Center for Disease Control data, to ensure that their policing data is not contaminated by bias. Additionally, they take steps to understand the proper benchmarks for disparity research and to determine if there are any disparities in their practices based on race.

What measures can be taken to ensure equitable treatment of all racial groups within the criminal justice system?


**`Final Response:`** To ensure equitable treatment of all racial groups within the criminal justice system, it is important to establish appropriate benchmarks for assessing disparities, rather than relying solely on overall population metrics. One measure is to consider rates of police exposure based on criminal activity data from reliable sources. This approach can help in accurately analyzing and addressing any racial disparities that may exist in the criminal justice system. Additionally, ongoing research and analysis by experts can provide valuable insights into potential biases and disparities, leading to informed decision-making and potential reforms to promote fairness and equality for all racial groups.